First off make secure connection data bricks account to both raw and curated data lakes, we would only want this on through day ETLs for example

In [2]:
spark.conf.set("fs.azure.account.auth.type", "OAuth")
spark.conf.set("fs.azure.account.oauth.provider.type", "org.apache.hadoop.fs.azurebfs.oauth2.ClientCredsTokenProvider")
spark.conf.set("fs.azure.account.oauth2.client.id", "{clientid}")
spark.conf.set("fs.azure.account.oauth2.client.secret", "{clientsecret}")
spark.conf.set("fs.azure.account.oauth2.client.endpoint", "https://login.microsoftonline.com/{tenantid}/oauth2/token")
spark.conf.set("fs.azure.createRemoteFileSystemDuringInitialization", "true")
dbutils.fs.ls("abfss://test@griffvnetlk2.dfs.core.windows.net/")
dbutils.fs.ls("abfss://test@griffvnetlk2curated.dfs.core.windows.net/")
spark.conf.set("fs.azure.createRemoteFileSystemDuringInitialization", "false")

Then read into data frame from raw data lake

In [4]:
pathraw = "abfss://test@griffvnetlk2.dfs.core.windows.net/TestUpload.csv"

dfraw = (spark
  .read
  .option("header", True)
  .csv(pathraw)
)

display(dfraw)


id,char
1,a


Write into curated data lake as parquet

In [6]:
pathcurated = "abfss://test@griffvnetlk2curated.dfs.core.windows.net/TestUpload.parquet"

dfraw.write.option("header", "true").parquet(pathcurated)


--------------------------------------------------------------------------- 
 Py4JJavaError Traceback (most recent call last)
 /databricks/spark/python/pyspark/sql/utils.py in deco (*a, **kw) 
 62 try : 
 ---> 63 return f ( * a , ** kw ) 
 64 except py4j . protocol . Py4JJavaError as e : 

 /databricks/spark/python/lib/py4j-0.10.7-src.zip/py4j/protocol.py in get_return_value (answer, gateway_client, target_id, name) 
 327 "An error occurred while calling {0}{1}{2}.\n" . 
 --> 328 format(target_id, ".", name), value)
 329 else : 

 Py4JJavaError : An error occurred while calling o408.parquet.
: org.apache.spark.sql.AnalysisException: path abfss://test@griffvnetlk2curated.dfs.core.windows.net/TestUpload.parquet already exists.;
	at org.apache.spark.sql.execution.datasources.InsertIntoHadoopFsRelationCommand.run(InsertIntoHadoopFsRelationCommand.scala:148)
	at org.apache.spark.sql.execution.command.DataWritingCommandExec.sideEffectResult$lzycompute(commands.scala:108)
	at org.apache.spark.sql.execution.command.DataWritingCommandExec.sideEffectResult(commands.scala:106)
	at org.apache.spark.sql.execution.command.DataWritingCommandExec.doExecute(commands.scala:126)
	at org.apache.spark.sql.execution.SparkPlan$$anonfun$execute$1.apply(SparkPlan.scala:146)
	at org.apache.spark.sql.execution.SparkPlan$$anonfun$execute$1.apply(SparkPlan.scala:134)
	at org.apache.spark.sql.execution.SparkPlan$$anonfun$executeQuery$5.apply(SparkPlan.scala:187)
	at org.apache.spark.rdd.RDDOperationScope$.withScope(RDDOperationScope.scala:151)
	at org.apache.spark.sql.execution.SparkPlan.executeQuery(SparkPlan.scala:183)
	at org.apache.spark.sql.execution.SparkPlan.execute(SparkPlan.scala:134)
	at org.apache.spark.sql.execution.QueryExecution.toRdd$lzycompute(QueryExecution.scala:115)
	at org.apache.spark.sql.execution.QueryExecution.toRdd(QueryExecution.scala:115)
	at org.apache.spark.sql.DataFrameWriter$$anonfun$runCommand$1.apply(DataFrameWriter.scala:710)
	at org.apache.spark.sql.DataFrameWriter$$anonfun$runCommand$1.apply(DataFrameWriter.scala:710)
	at org.apache.spark.sql.execution.SQLExecution$$anonfun$withCustomExecutionEnv$1.apply(SQLExecution.scala:111)
	at org.apache.spark.sql.execution.SQLExecution$.withSQLConfPropagated(SQLExecution.scala:240)
	at org.apache.spark.sql.execution.SQLExecution$.withCustomExecutionEnv(SQLExecution.scala:97)
	at org.apache.spark.sql.execution.SQLExecution$.withNewExecutionId(SQLExecution.scala:170)
	at org.apache.spark.sql.DataFrameWriter.runCommand(DataFrameWriter.scala:710)
	at org.apache.spark.sql.DataFrameWriter.saveToV1Source(DataFrameWriter.scala:306)
	at org.apache.spark.sql.DataFrameWriter.save(DataFrameWriter.scala:292)
	at org.apache.spark.sql.DataFrameWriter.save(DataFrameWriter.scala:235)
	at org.apache.spark.sql.DataFrameWriter.parquet(DataFrameWriter.scala:600)
	at sun.reflect.NativeMethodAccessorImpl.invoke0(Native Method)
	at sun.reflect.NativeMethodAccessorImpl.invoke(NativeMethodAccessorImpl.java:62)
	at sun.reflect.DelegatingMethodAccessorImpl.invoke(DelegatingMethodAccessorImpl.java:43)
	at java.lang.reflect.Method.invoke(Method.java:498)
	at py4j.reflection.MethodInvoker.invoke(MethodInvoker.java:244)
	at py4j.reflection.ReflectionEngine.invoke(ReflectionEngine.java:380)
	at py4j.Gateway.invoke(Gateway.java:295)
	at py4j.commands.AbstractCommand.invokeMethod(AbstractCommand.java:132)
	at py4j.commands.CallCommand.execute(CallCommand.java:79)
	at py4j.GatewayConnection.run(GatewayConnection.java:251)
	at java.lang.Thread.run(Thread.java:748)


During handling of the above exception, another exception occurred:

 AnalysisException Traceback (most recent call last)
 <command-2568288666873843> in <module> 
 1 pathcurated = "abfss://test@griffvnetlk2curated.dfs.core.windows.net/TestUpload.parquet" 
 2 
 ----> 3 dfraw . write . option ( "header" , "true" ) . parquet ( pathcurated ) 

 /databricks/spark/python/pyspark/sql/readwriter.py in parquet (self, path, mode, partitionBy, compression) 
 842 self

read from curated data lake into dataframe to prove moved and in different format

In [8]:
dfcurated = (spark
  .read
  .option("header", True)
  .parquet(pathcurated)
)

display(dfcurated)

id,char
1,a


place data into a databricks table example and can query like sql, see the structure under data on the left

In [10]:
%sql 

DROP TABLE IF EXISTS test;

CREATE TABLE test USING parquet OPTIONS (path "abfss://test@griffvnetlk2curated.dfs.core.windows.net/TestUpload.parquet");

SELECT * FROM test;

id,char
1,a
